# Michelin Project

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import time
import timeit
from IPython.core.display import Image
import numpy as np
from matplotlib import pyplot as plt
import cPickle

% matplotlib inline

#!pip install -U textacy
#import textacy

In [2]:
Image(url='http://bit.ly/29EoYnx', width=350)
Image(url='http://bit.ly/29Pzmae', width=350)

import statsmodels.api as st

import sklearn

from sklearn import linear_model as lm

from sklearn import cross_validation, preprocessing

import matplotlib.cm as cm

from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_curve

from sklearn.metrics import roc_auc_score

from sklearn.cross_validation import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.metrics import precision_score

from sklearn.metrics import recall_score

from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import cross_val_predict

from sklearn.cross_validation import cross_val_score

import matplotlib.patheffects as path_effects

from sklearn.metrics import mean_squared_error

Creates a list of path extension for each restaurant site (saves as dataframe):

In [3]:
# big_URL = 'http://www.viamichelin.com/web/Restaurants?geoboundaries=-59.1759282,-82.96875:82.166446,55.546875'
# page_URL = '&page='

# paths = []
# for page in range(2,775):
#     pre_mich = requests.get(big_URL + page_URL + str(page))
#     pre_soup = BeautifulSoup(pre_mich.content, "lxml")
#     for restaurant in pre_soup.find_all('a', href=re.compile('/web/Restaurant/')):
#         path = restaurant["href"]
#         paths.append(path)

# paths = pd.DataFrame(paths)
# paths.columns = ['path']
# paths.to_csv('assets/michelin_paths___RENAME_ME____.csv', encoding='utf-8')

Load the above dataframe from file:

In [4]:
URL = 'http://www.viamichelin.com'
paths = pd.read_csv('./assets/michelin_paths.csv')
paths = paths['0']
paths.reset_index(drop = True)
paths.columns = ['path']

A subset of this list for BeautifulSoup/cleaning trials:

In [5]:
test_paths = paths[0:10]

Scrape each restaurant site, append content to list, save pickled list to file (**10+hr, incl. writing to file**):

In [6]:
# big_soup = []
# for i in paths:
#     time.sleep(1)
#     tiny_request = requests.get(URL + i)
#     if str(tiny_request) != '<Response [200]>':
#         print URL + i
#     tiny_soup = BeautifulSoup(tiny_request.content, "lxml")
#     big_soup___DO_NOT_OVERWRITE___.append(tiny_soup)

# cPickle.dump(big_soup, open('big_soup___DO_NOT_OVERWRITE___.p', 'wb')) 

Previous scrape terminated early, finish the scrape (hence big_soup_pt1, big_soup_pt2):

In [7]:
# next_paths = paths[9857:16751]

# big_soup__NAME__ = []
# for i in next_paths:
#     time.sleep(1)
#     tiny_request = requests.get(URL + i)
#     if str(tiny_request) != '<Response [200]>':
#         print URL + i
#     tiny_soup = BeautifulSoup(tiny_request.content, "lxml")
#     big_soup__NAME__.append(tiny_soup)

Prints hyperlink for associated path extension, use to view pages spat out above:

In [8]:
lil_extension = '/web/Restaurant/_______________'
for extension in paths:
    if extension == lil_extension:
        print paths[paths == lil_extension]

Non-200 response pages from scrape during week of July 11, 2016 (run individually or add manually to dataframe):


from big_soup_pt1.p scrape (July 14, 2016):

http://www.viamichelin.com/web/Restaurant/LEINSWEILER-76829-Leinsweiler_Hof-82259-41102

http://www.viamichelin.com/web/Restaurant/Landau_Godramstein-76829-Westphals_Kulinarium-117644-41102

http://www.viamichelin.com/web/Restaurant/South_Kensington-SW5_0LJ-Cambio_de_Tercio-123727-41102

http://www.viamichelin.com/web/Restaurant/Tours-37000-Le_Theleme-273277-41102

http://www.viamichelin.com/web/Restaurant/PARADISO-33050-Al_Paradiso-145932-41102

from big_soup_pt2.p scrape (July 15, 2016):

http://www.viamichelin.com/web/Restaurant/New_York-10011-Salinas-336504-41102

http://www.viamichelin.com/web/Restaurant/Saint_Etienne-42000-Aromatic_Pierre_Daret-438788-41102

In [9]:
# next_paths = paths[9857:16751]

# big_soup2 = []
# for i in next_paths:
#     time.sleep(1)
#     tiny_request = requests.get(URL + i)
#     if str(tiny_request) != '<Response [200]>':
#         print URL + i
#     tiny_soup = BeautifulSoup(tiny_request.content, "lxml")
#     big_soup2.append(tiny_soup)

In [10]:
# cPickle.dump(big_soup__RENAME_ME__, open('big_soup__RENAME_ME__.p', 'wb')) 

In [ ]:
big_soup_pt1 = cPickle.load(open('/Users/Rebecca/Desktop/big_soup_pt1.p', 'rb'))
#big_soup_pt2 = cPickle.load(open('/Users/Rebecca/Desktop/big_soup_pt2.p', 'rb'))
print len(big_soup_pt1)
#print len(big_soup_pt2)

In [ ]:
#big_soup = pd.concat([big_soup_pt1, big_soup_pt2], axis = 0)

In [5]:
def restaurantify(list_soup):
    
    '''This function creates a dataframe with restaurant observations and columns containing relevant information.'''
    
    name = []
    address = []
    cuisine = []
    blurb = []
    price = []
    author = []
    distinction = []
    photo = []
    add_info = []


    for tiny_soup in list_soup:
        
        if tiny_soup.findAll('div', class_ = 'datasheet-item datasheet-name') is None:
            name.append('empty')
            continue
        else:
            for rest_name in tiny_soup.findAll('div', class_ = 'datasheet-item datasheet-name'):
                name.append(rest_name.text)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-item') is None:
            address.append('empty')
            continue
        else:    
            addresses = []
            for rest_address in tiny_soup.findAll('div', class_ = 'datasheet-item'):
                addresses.append(rest_address.text)
            if len(addresses) >= 2:
                address.append(addresses[1])
            else:
                address.append('weird_address')
        
        if tiny_soup.findAll('div', class_ = 'datasheet-cooking-type') is None:
            cuisine.append('empty')
            continue
        else:
            for rest_cuisine in tiny_soup.findAll('div', class_ = 'datasheet-cooking-type'):
                cuisine.append(rest_cuisine.text)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-description') is None:
            blurb.append('empty')
            continue
        else:
            for rest_blurb in tiny_soup.findAll('div', class_ = 'datasheet-description'):
                for j in rest_blurb.find_all('blockquote'):
                    blurb.append(j.text)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-price') is None:
            price.append('empty')
            continue
        else:
            for rest_price in tiny_soup.findAll('div', class_ = 'datasheet-price'):
                price.append(rest_price.text)
        
        if tiny_soup.findAll('p', class_ = 'michelin-guide-inspectors') is None:
            author.append('empty')
            continue
        else:
            for rest_author in tiny_soup.findAll('p', class_ = 'michelin-guide-inspectors'):
                author.append(rest_author.text)
        
        if tiny_soup.findAll('div', class_ = 'poi-item-stars') is None:
            distinction.append('empty')
            continue
        else:
            distinctions = []
            for rest_distinction in tiny_soup.findAll('div', class_ = 'poi-item-stars'):
                for j in rest_distinction.find_all('span'):
                    distinctions.append(j['class'])
            distinction.append(distinctions)
        
        if tiny_soup.findAll('div', class_ = 'datasheet-more-info datasheet-photo clearfx') is None:
            photo.append('empty')
            continue
        else:
            for rest_photo in tiny_soup.findAll('div', class_ = 'datasheet-more-info datasheet-photo clearfx'):
                photos = []
                if rest_photo.findAll('p') is None:
                    photo.append('empty')
                else:
                    for number in rest_photo.findAll('p'):
                        photos.append(number.text)
                photo.append(photos)

        if tiny_soup.findAll('div', class_ = 'datasheet-more-info clearfx') is None:
            info.append('empty')
            continue
        else:
            infos = []
            for rest_add_info in tiny_soup.findAll('div', class_ = 'datasheet-more-info clearfx'):
                infos.append(rest_add_info.text)
            add_info.append(infos)
        
    df_nice = pd.DataFrame(zip(name, cuisine, blurb, price, author),\
                        columns = ['name', 'cuisine', 'blurb', 'price', 'author']) 
    
    df_5long = pd.DataFrame(zip(address, distinction, add_info), columns = ['address', 'distinction', 'add_info'])
    
    df_photo = pd.DataFrame(photo, columns = ['photo'])
    
    print 'name: %s' %len(name)
    print 'address: %s' %len(address)
    print 'cuisine: %s' %len(cuisine)
    print 'blurb: %s' %len(blurb)
    print 'price: %s' %len(price)
    print 'author: %s' %len(author)
    print 'distinction: %s' %len(distinction)
    print 'photo: %s' %len(photo)
    print 'add_info: %s' %len(add_info)
    return df_nice, df_5long, df_photo, name, cuisine, blurb, price, author, address, distinction, add_info, photo

In [ ]:
restaurantify(big_soup_pt1)

In [ ]:
def get_name(df):
    df['name'] = df['name'].apply(lambda x: x.replace('\n\t', ''))
    df['name'] = df['name'].apply(lambda x: x.replace('\n', ''))
    return df

def split_price(df):
    df['price'] = df['price'].apply(lambda x: x.replace('\n\t\t\tFrom ', ''))
    df['price'] = df['price'].apply(lambda x: x.replace('\n\t\t\t', ' '))
    df['price'] = df['price'].apply(lambda x: x.replace('\n', ''))
    low_price = []
    high_price = []
    anything = []
    for pricerange in df['price']:
        pricerange = pricerange.split(' to ')
        low_price.append(pricerange[0])
        high_price.append(pricerange[1])
        if len(address) > 2:
            anything.append(pricerange[2])
    df['low_price'] = low_price        
    df['high_price'] = high_price
    if len(anything) == 0:
        print 'All addresses split correctly by comma!'
        del df['price']
    else: 
        print 'We have a problem!  Not all addresses split correctly by comma!'
    return df

def clean_cuisine(df):
    test_mich['cuisine'] = test_mich['cuisine'].apply(lambda x: x.replace('|', ' '))
    test_mich['cuisine'] = test_mich['cuisine'].apply(lambda x: x.replace('Cuisine', ''))
    test_mich['cuisine'] = test_mich['cuisine'].apply(lambda x: x.replace('cuisine', ''))
    return df

def split_address(df):
    street_address = []
    general_location = []
    anything = []
    for address in df['address']:
        address = address.split(',')
        street_address.append(address[0])
        general_location.append(address[1])
        if len(address) > 2:
            anything.append(address[2])
    df['street_address'] = street_address
    df['general_location'] = general_location
    if len(anything) == 0:
        print 'All addresses split correctly by comma!'
        del df['address']
    else:
        print 'We have a problem!  Not all addresses split correctly by comma!'
    return df

def clean_blurb(df):
    df['blurb'] = df['blurb'].apply(lambda x: x.replace('\n\nMICHELIN Guide\n\n\t\t\t', ''))
    return df

def get_standing(df):
    df['standing'] = df['standing'].apply(lambda x: x.replace('[\'standing-', ''))
    df['standing'] = df['standing'].apply(lambda x: x.replace('\']', ''))
    return df

# def get_distinction(df):
#     df['distinction'] = df['distinction'].apply(lambda x: x.replace('[\'', ''))
#     df['distinction'] = df['distinction'].apply(lambda x: x.replace('\']', ''))
#     return df

def get_photos(df):
    df['photos'] = df['photos'].apply(lambda x: x.replace('\nPhoto (', ''))
    df['photos'] = df['photos'].apply(lambda x: x.replace('\nPhotos (', ''))
    test_mich['photos'] = test_mich['photos'].apply(lambda x: x.replace(')', ''))
    test_mich['photos'] = test_mich['photos'].apply(lambda x: int(x.replace('\n', '')))
    return df

def get_where(df):
    location = []
    location_info = []
    lat_lon = []
    geolocator = Nominatim()
    for street in df['street_address']:
        location = geolocator.geocode(street)
        location.append(location.address)
        location_info.append(location.raw)
        lat_lon.append('(' + str(location.latitude) + ', ' + str(location.longitude) + ')')
    df['location'] = location
    df['lat_lon'] = lat_lon
    df['location_info'] = location_info
    return df

def how_far_to_people(df):
    nearest_people = []
    for i in range(len(df['town'])):
        eat = df['lat_lon'][i]
        people = df[''][i]
        too_far = great_circle(eat, people).kilometers
        nearest_people.append(too_far)
    df['nearest_people'] = nearest_people
    return df

def split_blurb(df):
    blurb = []
    not_blurb = []
#    for bigblurb in df['blurb']:
#     FILL IN   
    df['blurb'] = blurb
    return df

def blurb_count(df):
    word_count = []
    for i in df['blurb']:
        x = i.split(' ')
        count = len(x)
        word_count.append(count)
    df['word_count'] = word_count
    return df

def get_len_range(df):
    price_range = []
    for i in range(len(df)):
        pricerange = df['high_price'][i] - df['low_price'][i]
        price_range.append(pricerange)
    df['price_range'] = price_range
    return df

In [109]:
# 100 photos (max)
for i in range(len(mich)):
    if len(mich['photo'][i]) == 12:
        print mich.loc[i]

name                                              \n\tEtxanobe\n
address                        passeig de la Pau 27, 08600 Berga
cuisine                                   Cuisine Modern cuisine
blurb          \n\t\t\tEtxanobe is located on the top floor o...
price                    \n\t\t\tFrom 55 EUR\n\t\t\tto 100 EUR\n
author                                 MICHELIN guide inspectors
distinction                                      [[standing-13]]
photo                                               Photos (100)
add_info       [\nMeal prices\n\nMenu 22/48 € - Carte 31/58 €...
Name: 4883, dtype: object


In [107]:
#mich.to_csv('/Users/Rebecca/DSI-projects/projects/capstone/assets/michelin_dataframe_pt1.csv', index = False)

In [ ]:
for rest_author in big_soup2.findAll('p', class_= 'michelin-guide-inspectors'):
    if str(rest_author.text) != 'MICHELIN guide inspectors':
        print rest_author

7150

name: 9853

address: 9858

cuisine: 9853

blurb: 9853

price: 9853

author: 9853

distinction: 9858

standing: 9440

photo: 7150

add_info: 9858

In [82]:
length = []
for i in range(len(mich)):
    length.append(len(mich['add_info'][i]))
widdle = set(length)
print widdle

set([0, 2, 3, 4])


In [102]:
length = []
for i in range(len(mich)):
    length.append(len(mich['photo'][i]))
widdle = set(length)
print widdle

set([9, 10, 11, 12])


In [106]:
for i in range(len(mich)):
    if len(mich['photo'][i]) == 11:
        print mich.loc[i]

name                                                \n\tLe Cap\n
address        71 bd du Gén.-de-Gaulle, 06230 Saint-Jean-Cap-...
cuisine                                   Cuisine Modern cuisine
blurb          \n\t\t\tSet your course for a fine modern menu...
price                   \n\t\t\tFrom 158 EUR\n\t\t\tto 390 EUR\n
author                                 MICHELIN guide inspectors
distinction                      [[star], [star], [standing-21]]
photo                                                Photos (20)
add_info       [\nMeal prices\n\nMenu 158/248 € - Carte 172/3...
Name: 1, dtype: object
name                                          \n\tLes Pyrénées\n
address        19 pl. Charles-de-Gaulle, 64220 Saint-Jean-Pie...
cuisine                                  Cuisine Classic cuisine
blurb          \n\t\t\tAn institution in St Jean Pied de Port...
price                    \n\t\t\tFrom 42 EUR\n\t\t\tto 112 EUR\n
author                                 MICHELIN guide inspectors
di

In [100]:
photo_df = pd.DataFrame(mich['photo'])
photo_df.reset_index(drop = True)
for i in range(len(photo_df)):
    if len(photo_df.loc[i]) == 12:
        print i

In [101]:
photo_df.loc[0:10]

,photo
0,Photos (9)
1,Photos (20)
2,Photos (7)
3,Photo (1)
4,Photos (20)
5,Photo (1)
6,Photos (6)
7,Photos (6)
8,Photo (1)
9,Photo (1)


In [58]:
test = pd.DataFrame(mich['standing'])
test.head()

,standing
0,[standing-13]
1,[standing-21]
2,[standing-19]
3,[standing-13]
4,[standing-14]


In [77]:
mich.loc[7149]

name                                                \n\tMedium\n
address        Hofenstetten 55, 92431 Neunburg vorm Wald-Hofe...
cuisine                                    Cuisine International
blurb          \n\t\t\tDiners at Medium appreciate the modern...
price                     \n\t\t\tFrom 33 EUR\n\t\t\tto 64 EUR\n
author                                 MICHELIN guide inspectors
distinction                      [[star], [star], [standing-19]]
photo                                                 Photos (2)
add_info       [\nMeal prices\n\nMenu 80/138 €\n\n, \nGood to...
Name: 7149, dtype: object

Scrape and organize data using restaurantify() on list of site paths:

In [179]:
def get_where(df):
    location = []
    location_info = []
    lat_lon = []
    geolocator = Nominatim()
    for street in df['street_address']:
        location = geolocator.geocode(street)
        location.append(location.address)
        location_info.append(location.raw)
        lat_lon.append('(' + str(location.latitude) + ', ' + str(location.longitude) + ')')
    df['location'] = location
    df['lat_lon'] = lat_lon
    df['location_info'] = location_info
    return df

In [10]:
#####  TESTING CELL

pre_request = requests.get('http://www.viamichelin.com/' + '')
pre_soup = BeautifulSoup(pre_request.content, 'lxml')

mess_list = []
for mess in pre_soup.find_all('div', class_ = 'datasheet-item'):
    print type(mess)
#    mess_list.append(mess[0].text.encode('utf8'))

In [17]:
test_mich = restaurantify(test_paths)

In [18]:
test_mich.head(2)

,little_mess,cuisine,price,blurb,author,distinction,standing,photos,add_info


In [19]:
test_paths

0    /web/Restaurant/Penmarch-29760-Sterenn-12822-4...
1    /web/Restaurant/Saint_Jean_Cap_Ferrat-06230-Le...
2    /web/Restaurant/Saint_Jean_Cap_Ferrat-06230-La...
3    /web/Restaurant/St_Jean_du_Bruel-12230-Midi_Pa...
4    /web/Restaurant/Saint_Jean_Pied_de_Port-64220-...
5    /web/Restaurant/Saint_Laurent_de_Mure-69720-Ch...
6    /web/Restaurant/Saint_Leonard_de_Noblat-87400-...
7    /web/Restaurant/St_Malo-35400-La_Grassinais-12...
8    /web/Restaurant/Tilques-62500-Chateau_Tilques-...
9    /web/Restaurant/Saint_Pons-07580-Hostellerie_G...
Name: 0, dtype: object

In [11]:
geolocator = Nominatim()
location = geolocator.geocode('Route d\'Eckmuhl')
location

Location((47.7982044, -4.37276110875, 0.0))

In [27]:
URL = 'http://www.viamichelin.com'
little_mess = []
for i in test_paths:
    tiny_request = requests.get(URL + i)
    if str(tiny_request) != '<Response [200]>':
        print tiny_request
    tiny_soup = BeautifulSoup(tiny_request.content, "lxml")
    for mess in tiny_soup.find_all('div', class_ = 'datasheet-item'):
            little_mess.append(str(mess.text.encode('utf8')))
print little_mess[3]
print '\n'


	Le Cap





In [19]:
def add(x, y):
    z = x + y
    return z
%timeit for i in range(10): add(10, 5)


100000 loops, best of 3: 3.22 µs per loop


In [ ]:
big_soup = []
no_soup = []
for i in paths:
    time.sleep(1)
    tiny_request = requests.get(URL + i)
    if str(tiny_request) != '<Response [200]>':
        print URL + i
        no_soup.append(i)
    tiny_soup = BeautifulSoup(tiny_request.content, "lxml")
    big_soup.append(tiny_soup)
big_soup.to_csv('big_soup.csv', encoding='utf-8')

Stream documents from disk and into a processed corpus:

In [ ]:
docs = textacy.corpora.fetch_bernie_and_hillary()
content_stream, metadata_stream = textacy.fileio.split_content_and_metadata(docs, 'text', itemwise=False)
corpus = textacy.TextCorpus.from_texts('en', content_stream, metadata_stream, n_threads=2)
print(corpus)

Represent corpus as a document-term matrix, with flexible weighting and filtering:

In [ ]:
doc_term_matrix, id2term = corpus.as_doc_term_matrix((doc.as_terms_list(words=True, ngrams=False, named_entities=True)\ 
                                for doc in corpus), weighting='tfidf', normalize=True, smooth_idf=True, min_df=2,\
                                    max_df=0.95)
print(repr(doc_term_matrix))

Train and interpret a topic model:

In [ ]:
model = textacy.tm.TopicModel('nmf', n_topics=10)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
print(doc_topic_matrix.shape)
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=10):
    print('topic', topic_idx, ':', '   '.join(top_terms))

Indexing, selection of documents in corpus:

In [ ]:
bernie_docs = list(corpus.get_docs(lambda doc: doc.metadata['speaker'] == 'Bernard Sanders'))
print(len(bernie_docs))
doc = corpus[-1]
print(doc)

Preprocess plain text, or highlight particular terms in it:

In [ ]:
textacy.preprocess_text(doc.text, lowercase=True, no_punct=True)[:70]
textacy.text_utils.keyword_in_context(doc.text, 'nation', window_width=35)

In [75]:
count = 0
for photo in big_soup[0].findAll('div', class_= 'datasheet-more-info datasheet-photo clearfx'):
    for small in photo.findAll('p'):
        print small.text
        print '\n'
        count +=1

Photos (9)




In [110]:
len(big_soup)

9858

In [111]:
type(big_soup)

list

In [112]:
big_soup[-1]

<!DOCTYPE html>\n<html lang="en" xmlns:v="urn:schemas-microsoft-com:vml">\n<head>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta charset="unicode-escape"/>\n<meta content="#1095f9" name="theme-color"/>\n<title>Eleven Rio\xa0- Lagoa\xa0:\xa0a Michelin Guide restaurant</title>\n<meta content="Eleven Rio: Michelin Guide review, users review, type of cuisine, opening times, meal prices .." name="description"/>\n<meta content="width=device-width, initial-scale=1.0, user-scalable=no" name="viewport"/>\n<meta content="yes" name="apple-mobile-web-app-capable"/>\n<meta content="app-id=443142682" name="apple-itunes-app"/>\n<meta content="app-id=com.viamichelin.android.viamichelinmobile" name="google-play-app"/>\n<meta content="http://www.viamichelin.com/logo-social.png" property="og:image"/>\n<meta content="Eleven Rio\xa0- Lagoa\xa0:\xa0a Michelin Guide restaurant" property="og:title"/>\n<meta content="Eleven Rio: Michelin Guide review, users review, type of cuisine, opening time

In [113]:
for rest_name in big_soup[-1].findAll('div', class_ = 'datasheet-item datasheet-name'):
    print rest_name.text


	Eleven Rio



In [116]:
## first restaurantify of paths went up to here:
lil_extension = '/web/Restaurant/Rio_de_Janeiro-22470_210-Eleven_Rio-467177-41102' 
for extension in paths:
    if extension == lil_extension:
        print paths[paths == lil_extension]

9857    /web/Restaurant/Rio_de_Janeiro-22470_210-Eleve...
Name: 0, dtype: object


In [117]:
len(paths)

16752

In [118]:
next_paths = paths[9857:16751]

In [11]:
for rest_photo in big_soup[1].findAll('div', class_ = 'datasheet-more-info datasheet-photo clearfx'):
     print rest_photo

NameError: name 'big_soup' is not defined